# Facility-by-Facility Comparison: TLO Model vs ANC Disruption Data

Matches TLO model-estimated disruption rates directly to ANC-derived disruption probabilities
at the individual facility level via `RealFacility_ID`.

**Structure:**
1. Configuration & imports
2. Load ANC disruption data (ground truth)
3. Build `Facility_ID` → `RealFacility_ID` mapping from weather event logs
4. Extract TLO disrupted counts by `RealFacility_ID` (climate run, all 200 draws)
5. Extract TLO baseline denominator in `RealFacility_ID` space
6. Compute per-facility disruption fractions & merge with ANC
7. Scatter: TLO model output vs ANC input, facility-level
8. Facility-level time series panels (top/bottom disrupted)
9. Summary statistics & residual analysis

In [20]:
# ══════════════════════════════════════════════════════════════════════════════
#  0.  CONFIGURATION  — edit paths here
# ══════════════════════════════════════════════════════════════════════════════
from pathlib import Path

# TLO simulation output folders
RESULTS_CLIMATE  = Path('/Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/climate_scenario_runs_lhs_param_scan-2026-02-03T101537Z')
RESULTS_BASELINE = Path('/Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-05T113912Z')

# ANC / climate data files
SSP     = 'ssp245'
MODEL   = 'mean'
SERVICE = 'ANC'

ANC_PREDICTIONS_CSV = Path(f'/Users/rem76/Desktop/Climate_Change_Health/Data/weather_predictions_with_X_{SSP}_{MODEL}_{SERVICE}.csv')
DISRUPTIONS_CSV     = Path(f'/Users/rem76/PycharmProjects/TLOmodel/resources/climate_change_impacts/ResourceFile_Precipitation_Disruptions_{SSP}_{MODEL}.csv')
TLO_MFL_CSV         = Path('/Users/rem76/PycharmProjects/TLOmodel/resources/healthsystem/organisation/ResourceFile_Master_Facilities_List.csv')

# Where to write figures
OUTPUT_DIR = Path('/Users/rem76/Desktop/Climate_Change_Health')
OUTPUT_DIR.mkdir(exist_ok=True)

# Analysis window
min_year   = 2025
max_year   = 2036   # exclusive
spacing_of_years = 1
# Analysis params
climate_sensitivity_analysis = False
main_text = True
parameter_uncertainty_analysis = False

if parameter_uncertainty_analysis:
    scenario_names = range(0,  200, 1)
    scenarios_of_interest = scenario_names
    suffix = "parameter_UA"
if main_text:
    scenario_names = [
        "Baseline",
        "Best Case",
        "Worst Case"
    ]
    suffix = "main_text"
    scenarios_of_interest = [0, 1, 2]

# Draw/folder 0 in the climate run is the no-disruption baseline — skip it
FIRST_CLIMATE_DRAW = 1   # draws 1 … N_DRAWS-1 are the actual LHS climate runs

# District name harmonisation  (split cities → parent district)
DISTRICT_MERGE = {
    'Mzimba North': 'Mzimba', 'Mzimba South': 'Mzimba',
    'Blantyre City': 'Blantyre', 'Zomba City': 'Zomba',
    'Lilongwe City': 'Lilongwe',
}

In [7]:
# ══════════════════════════════════════════════════════════════════════════════
#  1.  IMPORTS
# ══════════════════════════════════════════════════════════════════════════════
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy import stats as scipy_stats
from collections import defaultdict

from tlo import Date
from tlo.analysis.utils import extract_results, summarize

TARGET_YEARS = list(range(MIN_YEAR, MAX_YEAR))


Analysis years : 2025 – 2035  (11 years)
Parameter draws: 3


## 2. Load ANC disruption data

In [8]:
# ── ANC resource file: disruption probability by facility / year / month ──────
disruptions_df = pd.read_csv(DISRUPTIONS_CSV)
disruptions_df = disruptions_df.rename(columns={'Unnamed: 0': 'index_col'})

# Annual mean disruption probability per facility  (average across 12 months)
anc_annual = (
    disruptions_df
    .groupby(['RealFacility_ID', 'year'])['disruption']
    .mean()
    .reset_index()
    .rename(columns={'disruption': 'anc_prob', 'year': 'Year'})
)
anc_annual['anc_pct'] = anc_annual['anc_prob'] * 100

# Restrict to analysis window
anc_annual = anc_annual[anc_annual['Year'].between(min_year, max_year - 1)]

print(f'ANC data: {anc_annual["RealFacility_ID"].nunique()} unique facilities, '
      f'{anc_annual["Year"].nunique()} years')
print(f'  Mean annual disruption probability: {anc_annual["anc_pct"].mean():.2f}%')
anc_annual.head()

ANC data: 328 unique facilities, 11 years
  Mean annual disruption probability: 0.56%


,RealFacility_ID,Year,anc_prob,anc_pct
0,Area 30 Police Clinic,2025,0.006528,0.652783
1,Area 30 Police Clinic,2026,0.003169,0.316924
2,Area 30 Police Clinic,2027,0.002392,0.239246
3,Area 30 Police Clinic,2028,0.010278,1.027783
4,Area 30 Police Clinic,2029,0.010013,1.001276


In [9]:
# ── ANC predictions file: facility metadata and expected attendance ────────────
# (used for district / facility-type enrichment, not strictly needed for matching)
anc_pred = pd.read_csv(ANC_PREDICTIONS_CSV)
anc_pred = anc_pred.rename(columns={'Facility_ID': 'RealFacility_ID'})
anc_pred['District'] = anc_pred['District'].replace(DISTRICT_MERGE)

# Build a per-facility metadata table: one row per RealFacility_ID
fac_meta = (
    anc_pred[['RealFacility_ID', 'District', 'Facility_Type']]
    .drop_duplicates(subset='RealFacility_ID')
    .set_index('RealFacility_ID')
)

print(f'Facility metadata loaded: {len(fac_meta)} facilities')

Facility metadata loaded: 329 facilities


# Functions to extract data

In [21]:
target_year_sequence = range(min_year, max_year, spacing_of_years)

def get_num_treatments_total_delayed(_df):
    """Count total number of delayed HSI events"""
    _df["date"] = pd.to_datetime(_df["date"])
    _df = _df.loc[_df["date"].between(*TARGET_PERIOD)]
    return pd.Series(len(_df), name="total")

def get_num_treatments_total_cancelled(_df):
    """Count total number of cancelled HSI events"""
    _df["date"] = pd.to_datetime(_df["date"])
    _df = _df.loc[_df["date"].between(*TARGET_PERIOD)]
    return pd.Series(len(_df), name="total")

def get_num_treatments_total(_df):
    """Sum all treatment counts"""
    _df["date"] = pd.to_datetime(_df["date"])
    _df = _df.loc[_df["date"].between(*TARGET_PERIOD)]
    total = {}
    for d in _df["hsi_event_key_to_counts"]:
        for k, v in d.items():
            total[k] = total.get(k, 0) + v
    return pd.Series(sum(total.values()), name="total")

def get_num_dalys_total(_df):
    """Return total number of DALYS (Stacked) by label (total by age-group within the TARGET_PERIOD)"""
    return pd.Series(_df \
        .loc[_df.year.between(*[i.year for i in TARGET_PERIOD])] \
        .drop(columns=['date', 'sex', 'age_range', 'year']) \
        .sum().sum(), name="total")

def get_num_dalys_by_month(_df):
    """Sum all DALYs across all causes by month for the target year"""
    _df["date"] = pd.to_datetime(_df["date"])
    _df = _df.loc[_df["date"].between(*TARGET_PERIOD)]
    disease_columns = [col for col in _df.columns
                      if col not in ['age_range', 'month', 'sex', 'year', 'date']]
    return _df.groupby('month')[disease_columns].sum().sum(axis=1)

def get_population_for_year(_df):
    """Returns the population in the year of interest"""
    _df["date"] = pd.to_datetime(_df["date"])
    filtered_df = _df.loc[_df["date"].between(*TARGET_PERIOD)]
    numeric_df = filtered_df.drop(columns=["female", "male"], errors="ignore")
    return numeric_df.sum(numeric_only=True)

def get_num_treatments_by_real_facility_delayed(_df):
    """Count delayed HSI events grouped by RealFacility_ID"""
    _df["date"] = pd.to_datetime(_df["date"])
    _df = _df.loc[_df["date"].between(*TARGET_PERIOD)]
    _df = _df[_df["RealFacility_ID"].notna() & (_df["RealFacility_ID"] != "unknown")]
    if len(_df) == 0:
        return pd.Series(dtype=int)
    return _df.groupby("RealFacility_ID").size()

def get_num_treatments_by_real_facility_cancelled(_df):
    """Count cancelled HSI events grouped by RealFacility_ID"""
    _df["date"] = pd.to_datetime(_df["date"])
    _df = _df.loc[_df["date"].between(*TARGET_PERIOD)]
    _df = _df[_df["RealFacility_ID"].notna() & (_df["RealFacility_ID"] != "unknown")]
    if len(_df) == 0:
        return pd.Series(dtype=int)
    return _df.groupby("RealFacility_ID").size()

In [24]:
results_folder_baseline = Path(
    '/Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-20T161931Z')

In [23]:
# Storage dictionaries
all_scenarios_appointment_delayed_mean = {}
all_scenarios_appointment_cancelled_mean = {}
all_scenarios_dalys_mean = {}

# Get denominators in RealFacility_ID space
baseline_hsi_by_facility = {}

for target_year in target_year_sequence:
    TARGET_PERIOD = (Date(target_year, 1, 1), Date(target_year, 12, 31))

    hsi_by_facility = summarize(extract_results(
        results_folder_baseline,
        module='tlo.methods.healthsystem.summary',
        key='hsi_event_counts_by_facility_monthly',
        custom_generate_series=get_hsi_counts_by_facility_monthly,
        do_scaling=False
    ), only_mean=True, collapse_columns=False)[0]

    # Convert integer Facility_ID index → RealFacility_ID string index
    hsi_by_facility.index = pd.Index([int(x) for x in hsi_by_facility.index], name='Facility_ID')
    hsi_by_facility.index = hsi_by_facility.index.map(fac_id_to_real_s)
    hsi_by_facility = hsi_by_facility[hsi_by_facility.index.notna()]
    hsi_by_facility = hsi_by_facility.groupby(level=0).sum()
    hsi_by_facility.index.name = 'RealFacility_ID'
    baseline_hsi_by_facility[target_year] = hsi_by_facility

# Main loop — skip draw 0 (no-disruption baseline folder)
for draw in scenarios_of_interest:
    print(draw)
    all_years_data_delayed_mean = {}
    all_years_data_cancelled_mean = {}
    all_years_dalys_mean = {}

    for target_year in target_year_sequence:
        TARGET_PERIOD = (Date(target_year, 1, 1), Date(target_year, 12, 31))

        # Get delayed counts by RealFacility_ID from climate folder (NUMERATOR)
        num_delayed_by_facility = summarize(extract_results(
            results_folder_climate,
            module='tlo.methods.healthsystem.summary',
            key='Weather_delayed_HSI_Event_full_info',
            custom_generate_series=get_num_treatments_by_real_facility_delayed,
            do_scaling=False
        ), only_mean=True, collapse_columns=False)[draw]

        # Get cancelled counts by RealFacility_ID from climate folder (NUMERATOR)
        num_cancelled_by_facility = summarize(extract_results(
            results_folder_climate,
            module='tlo.methods.healthsystem.summary',
            key='Weather_cancelled_HSI_Event_full_info',
            custom_generate_series=get_num_treatments_by_real_facility_cancelled,
            do_scaling=False
        ), only_mean=True, collapse_columns=False)[draw]

        # Align and divide by baseline
        baseline_aligned, delayed_aligned = baseline_hsi_by_facility[target_year].align(num_delayed_by_facility, fill_value=0)
        delayed_proportions = delayed_aligned / baseline_aligned

        baseline_aligned, cancelled_aligned = baseline_hsi_by_facility[target_year].align(num_cancelled_by_facility, fill_value=0)
        cancelled_proportions = cancelled_aligned / baseline_aligned

        all_years_data_delayed_mean[target_year] = delayed_proportions
        all_years_data_cancelled_mean[target_year] = cancelled_proportions

    all_scenarios_appointment_delayed_mean[draw] = all_years_data_delayed_mean
    all_scenarios_appointment_cancelled_mean[draw] = all_years_data_cancelled_mean
    all_scenarios_dalys_mean[draw] = all_years_dalys_mean

NameError: name 'results_folder_baseline' is not defined

## 4. Extract TLO disrupted counts by `RealFacility_ID` — all draws

In [15]:
def _count_by_real_fac(_df):
    """Generic: count HSI events grouped by RealFacility_ID within TARGET_PERIOD."""
    print(_df)
    _df['date'] = pd.to_datetime(_df['date'])
    _df = _df.loc[_df['date'].between(*TARGET_PERIOD)]
    _df = _df[_df['RealFacility_ID'].notna() & (_df['RealFacility_ID'] != 'unknown')]
    if len(_df) == 0:
        return pd.Series(dtype=int)
    return _df.groupby('RealFacility_ID').size()


# ── Extract once per year for delayed and cancelled ───────────────────────────
# Result structures:
#   delayed_raw[year]   → DataFrame  shape (n_facilities, n_draws)
#   cancelled_raw[year] → DataFrame  shape (n_facilities, n_draws)

delayed_raw   = {}
cancelled_raw = {}

for year in TARGET_YEARS:
    TARGET_PERIOD = (Date(year, 1, 1), Date(year, 12, 31))
    print(f'  extracting year {year}', end='\r')

    delayed_raw[year] = extract_results(
        RESULTS_CLIMATE,
        module='tlo.methods.healthsystem.summary',
        key='Weather_delayed_HSI_Event_full_info',
        custom_generate_series=_count_by_real_fac,
        do_scaling=False,
    ).fillna(0)

    cancelled_raw[year] = extract_results(
        RESULTS_CLIMATE,
        module='tlo.methods.healthsystem.summary',
        key='Weather_cancelled_HSI_Event_full_info',
        custom_generate_series=_count_by_real_fac,
        do_scaling=False,
    ).fillna(0)

print('\nExtraction complete.')
print(f'  Example year {TARGET_YEARS[0]}: delayed shape = {delayed_raw[TARGET_YEARS[0]].shape}')

ValueError: All objects passed were None

## 5. Extract baseline denominator (total HSI events) in `RealFacility_ID` space

In [ ]:
def _hsi_by_facility_int(_df):
    """Sum HSI event counts by integer Facility_ID within TARGET_PERIOD."""
    _df['date'] = pd.to_datetime(_df['date'])
    _df = _df.loc[_df['date'].between(*TARGET_PERIOD)]
    if len(_df) == 0:
        return pd.Series(dtype=int)
    totals = defaultdict(int)
    for _, row in _df.iterrows():
        for key, val in row['counts'].items():
            if ':' in str(key):
                fac_id = int(key.split(':')[0])
                totals[fac_id] += val
    return pd.Series(totals)


baseline_by_real = {}  # year → pd.Series(index=RealFacility_ID, values=total HSI)

for year in TARGET_YEARS:
    TARGET_PERIOD = (Date(year, 1, 1), Date(year, 12, 31))
    print(f'  baseline year {year}', end='\r')

    raw_int = extract_results(
        RESULTS_BASELINE,
        module='tlo.methods.healthsystem.summary',
        key='hsi_event_counts_by_facility_monthly',
        custom_generate_series=_hsi_by_facility_int,
        do_scaling=False,
    ).fillna(0)

    # Average across baseline draws (columns) → one value per integer Facility_ID
    mean_int = raw_int.mean(axis=1)
    mean_int.index = mean_int.index.astype(int)

    # Map integer Facility_ID → RealFacility_ID
    mapped = mean_int.copy()
    mapped.index = mean_int.index.map(fac_id_to_real_s)
    mapped = mapped[mapped.index.notna()]          # drop unmapped facilities
    mapped = mapped.groupby(level=0).sum()         # sum if multiple TLO IDs → same name
    mapped.index.name = 'RealFacility_ID'
    baseline_by_real[year] = mapped

print('\nBaseline denominator ready.')
print(f'  Year {TARGET_YEARS[0]}: {len(baseline_by_real[TARGET_YEARS[0]])} facilities covered')

## 6. Compute per-facility disruption fractions & merge with ANC

In [ ]:
# ── Per draw, per year, per facility: disruption fraction ─────────────────────
#
# disruption_frac[year] → DataFrame  shape (n_facilities, n_draws)
#                          values = (delayed + cancelled) / baseline

disruption_frac = {}

for year in TARGET_YEARS:
    denom = baseline_by_real[year]              # Series  index=RealFacility_ID
    del_df  = delayed_raw[year]                 # DataFrame index=RealFacility_ID, cols=draws
    can_df  = cancelled_raw[year]

    # Align all three to the same index
    all_facs = denom.index.union(del_df.index).union(can_df.index)
    denom    = denom.reindex(all_facs, fill_value=0)
    del_df   = del_df.reindex(all_facs, fill_value=0)
    can_df   = can_df.reindex(all_facs, fill_value=0)

    total = del_df.add(can_df, fill_value=0)    # (delayed + cancelled) per draw

    # Divide row-wise by denominator; NaN where denominator is 0
    frac = total.divide(denom.replace(0, np.nan), axis=0)
    disruption_frac[year] = frac

print('Disruption fractions computed.')
print(f'  Example year {TARGET_YEARS[0]}: shape = {disruption_frac[TARGET_YEARS[0]].shape}')

In [ ]:
# ── Summarise across draws: mean / median / IQR per (facility, year) ──────────

summary_rows = []
for year in TARGET_YEARS:
    df = disruption_frac[year]
    s  = pd.DataFrame({
        'mean':   df.mean(axis=1),
        'median': df.median(axis=1),
        'p25':    df.quantile(0.25, axis=1),
        'p75':    df.quantile(0.75, axis=1),
        'p05':    df.quantile(0.05,  axis=1),
        'p95':    df.quantile(0.95,  axis=1),
    })
    s.index.name = 'RealFacility_ID'
    s = s.reset_index()
    s['Year'] = year
    summary_rows.append(s)

tlo_summary = pd.concat(summary_rows, ignore_index=True)
# Convert to %
for col in ['mean', 'median', 'p25', 'p75', 'p05', 'p95']:
    tlo_summary[col] *= 100

print(f'TLO summary table: {tlo_summary.shape[0]} rows  ({tlo_summary["RealFacility_ID"].nunique()} facilities × {len(TARGET_YEARS)} years)')

In [ ]:
# ── Merge TLO model summary with ANC disruption data on (RealFacility_ID, Year) ─

comparison = tlo_summary.merge(
    anc_annual,
    on=['RealFacility_ID', 'Year'],
    how='inner'
)

# Optionally enrich with facility metadata (district, type)
comparison = comparison.merge(
    fac_meta.reset_index(),
    on='RealFacility_ID',
    how='left'
)

# Drop facilities with no meaningful baseline counts in TLO model
# (mean TLO disruption NaN means the facility never appeared in the model outputs)
comparison = comparison.dropna(subset=['mean', 'anc_pct'])

n_fac   = comparison['RealFacility_ID'].nunique()
n_years = comparison['Year'].nunique()
print(f'Matched: {n_fac} facilities × {n_years} years = {len(comparison)} rows')
comparison[['RealFacility_ID', 'Year', 'mean', 'p25', 'p75', 'anc_pct', 'District', 'Facility_Type']].head(8)

## 7. Scatter plot: TLO model output vs ANC input, facility-level

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

years_in_data  = sorted(comparison['Year'].unique())
cmap           = plt.cm.plasma
norm           = Normalize(vmin=min(years_in_data), vmax=max(years_in_data))
axis_max       = max(comparison[['mean', 'anc_pct']].max()) * 1.1


# ─── Panel A: every facility-year point, coloured by year ────────────────────
ax = axes[0]
for year in years_in_data:
    sub = comparison[comparison['Year'] == year].dropna(subset=['mean', 'anc_pct'])
    ax.scatter(
        sub['anc_pct'], sub['mean'],
        c=[cmap(norm(year))] * len(sub),
        alpha=0.4, s=12, linewidths=0
    )

ax.plot([0, axis_max], [0, axis_max], 'k--', lw=1, alpha=0.5, label='1:1 line')

sub_all = comparison.dropna(subset=['mean', 'anc_pct'])
slope, intercept, r, p, _ = scipy_stats.linregress(sub_all['anc_pct'], sub_all['mean'])
xs = np.linspace(0, axis_max, 100)
ax.plot(xs, intercept + slope * xs, 'r-', lw=1.8,
        label=f'OLS  slope={slope:.2f}  R²={r**2:.3f}')

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax, label='Year', shrink=0.85)
ax.set_xlabel('ANC disruption probability (%)',  fontsize=10)
ax.set_ylabel('TLO model disruption rate (%)\n[mean across draws]', fontsize=10)
ax.set_title('A) All facility-years\n(coloured by year)', fontsize=11)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3, ls=':')


# ─── Panel B: time-averaged with IQR error bars, coloured by Facility_Type ───
ax = axes[1]
fac_avg = (
    comparison
    .groupby('RealFacility_ID')[['mean', 'p25', 'p75', 'anc_pct', 'Facility_Type']]
    .agg({'mean': 'mean', 'p25': 'mean', 'p75': 'mean',
          'anc_pct': 'mean', 'Facility_Type': 'first'})
    .reset_index()
    .dropna()
)

# Colour by facility type (broad groups)
type_colour = {
    'Health Post': '#1f77b4', 'Health Centre': '#ff7f0e', 'Hospital': '#2ca02c',
    'Other': '#9467bd'
}
def _broad_type(t):
    if pd.isna(t): return 'Other'
    t = t.lower()
    if 'hospital' in t: return 'Hospital'
    if 'health centre' in t or 'health center' in t: return 'Health Centre'
    if 'health post' in t or 'dispensary' in t or 'village' in t: return 'Health Post'
    return 'Other'

fac_avg['broad_type'] = fac_avg['Facility_Type'].apply(_broad_type)

for bt, grp in fac_avg.groupby('broad_type'):
    ax.errorbar(
        grp['anc_pct'], grp['mean'],
        yerr=[grp['mean'] - grp['p25'], grp['p75'] - grp['mean']],
        fmt='o', alpha=0.5, markersize=5, elinewidth=0.6, capsize=0,
        color=type_colour.get(bt, '#9467bd'), label=bt
    )

ax.plot([0, axis_max], [0, axis_max], 'k--', lw=1, alpha=0.5)
slope2, intercept2, r2, p2, _ = scipy_stats.linregress(
    fac_avg['anc_pct'], fac_avg['mean'])
ax.plot(xs, intercept2 + slope2 * xs, 'r-', lw=1.8,
        label=f'OLS  slope={slope2:.2f}  R²={r2**2:.3f}')

ax.set_xlabel('ANC disruption probability (%) [time-avg]', fontsize=10)
ax.set_ylabel('TLO model disruption rate (%) [time-avg mean]', fontsize=10)
ax.set_title('B) Per-facility averages\n(IQR bars, coloured by type)', fontsize=11)
ax.legend(fontsize=7, loc='upper left')
ax.grid(True, alpha=0.3, ls=':')


# ─── Panel C: residuals vs ANC disruption probability ─────────────────────────
ax = axes[2]
sub_all = comparison.dropna(subset=['mean', 'anc_pct']).copy()
sub_all['residual'] = sub_all['mean'] - sub_all['anc_pct']

ax.scatter(
    sub_all['anc_pct'], sub_all['residual'],
    c=[cmap(norm(y)) for y in sub_all['Year']],
    alpha=0.35, s=12, linewidths=0
)
ax.axhline(0, color='k', lw=1, ls='--', alpha=0.5, label='Zero residual')
ax.axhline(sub_all['residual'].mean(), color='r', lw=1.5,
           label=f'Mean residual = {sub_all["residual"].mean():.2f}%')

# Smooth trend in residuals (LOWESS-style via rolling)
bin_edges = np.percentile(sub_all['anc_pct'], np.arange(0, 101, 5))
sub_all['anc_bin'] = pd.cut(sub_all['anc_pct'], bins=bin_edges)
bin_median = sub_all.groupby('anc_bin')['residual'].median()
bin_centres = [interval.mid for interval in bin_median.index]
ax.plot(bin_centres, bin_median.values, 'orange', lw=2, zorder=5, label='Binned median')

sm2 = ScalarMappable(cmap=cmap, norm=norm)
sm2.set_array([])
fig.colorbar(sm2, ax=ax, label='Year', shrink=0.85)
ax.set_xlabel('ANC disruption probability (%)', fontsize=10)
ax.set_ylabel('TLO − ANC  (percentage-point residual)', fontsize=10)
ax.set_title('C) Residuals\n(TLO model minus ANC input)', fontsize=11)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3, ls=':')


fig.suptitle(
    f'Facility-level comparison: TLO model disruption vs ANC input disruption\n'
    f'n={n_fac} facilities | {N_DRAWS} parameter draws | {SSP.upper()} {MODEL} | {MIN_YEAR}–{MAX_YEAR-1}',
    fontsize=12, fontweight='bold', y=1.01
)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'facility_comparison_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'OLS all facility-years:  slope={slope:.3f}  R²={r**2:.3f}  p={p:.3g}')
print(f'OLS time-averaged facs:  slope={slope2:.3f}  R²={r2**2:.3f}  p={p2:.3g}')
print(f'Mean residual (TLO−ANC): {sub_all["residual"].mean():.2f}%')
print(f'Median |residual|:        {sub_all["residual"].abs().median():.2f}%')

## 8. Facility-level time-series panels

Show the 20 highest- and 20 lowest-disrupted facilities by ANC estimate.
Blue = TLO model median with IQR band. Purple = ANC disruption probability.

In [ ]:
# ── Rank facilities by mean ANC disruption probability ────────────────────────
fac_anc_rank = (
    anc_annual
    .groupby('RealFacility_ID')['anc_pct']
    .mean()
    .sort_values(ascending=False)
)

# Keep only facilities that also appear in the TLO outputs
tlo_facs = set(tlo_summary['RealFacility_ID'].unique())
fac_anc_rank = fac_anc_rank[fac_anc_rank.index.isin(tlo_facs)]

N_PANEL = 20
top_facs    = fac_anc_rank.head(N_PANEL).index.tolist()
bottom_facs = fac_anc_rank.tail(N_PANEL).index.tolist()
panel_facs  = top_facs + bottom_facs

print(f'Top    {N_PANEL}: mean ANC disruption = {fac_anc_rank.head(N_PANEL).mean():.1f}%')
print(f'Bottom {N_PANEL}: mean ANC disruption = {fac_anc_rank.tail(N_PANEL).mean():.1f}%')

In [ ]:
# Pre-build per-facility TLO time series (median + IQR across draws) ──────────
# Structure: fac_ts[facility] = dict with keys 'median', 'p25', 'p75'  (arrays of len n_years)

fac_ts = {}
DATE_INDEX = pd.to_datetime([f'{y}-07-01' for y in TARGET_YEARS])  # mid-year for plotting

for fac in panel_facs:
    med_arr, p25_arr, p75_arr = [], [], []
    for year in TARGET_YEARS:
        row = tlo_summary.loc[
            (tlo_summary['RealFacility_ID'] == fac) &
            (tlo_summary['Year'] == year)
        ]
        if len(row) == 0:
            med_arr.append(np.nan)
            p25_arr.append(np.nan)
            p75_arr.append(np.nan)
        else:
            med_arr.append(row['median'].values[0])
            p25_arr.append(row['p25'].values[0])
            p75_arr.append(row['p75'].values[0])
    fac_ts[fac] = {
        'median': np.array(med_arr),
        'p25':    np.array(p25_arr),
        'p75':    np.array(p75_arr),
    }

print('Time series arrays built.')

In [ ]:
n_cols = 8
n_rows = 5

fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(3.0 * n_cols, 2.5 * n_rows),
                         sharex=True, squeeze=True)
axes_flat = axes.flatten()

for i, fac in enumerate(panel_facs):
    ax    = axes_flat[i]
    group = 'Top' if i < N_PANEL else 'Bottom'
    title_colour = '#8B0000' if group == 'Top' else '#00008B'

    # TLO model
    ts = fac_ts[fac]
    ax.fill_between(DATE_INDEX, ts['p25'], ts['p75'],
                    color='#4169E1', alpha=0.25)
    ax.plot(DATE_INDEX, ts['median'],
            color='#4169E1', lw=1.5, label='TLO median')

    # ANC disruption probability
    anc_sub = anc_annual[anc_annual['RealFacility_ID'] == fac].sort_values('Year')
    if len(anc_sub) > 0:
        anc_dates = pd.to_datetime([f'{y}-07-01' for y in anc_sub['Year']])
        ax.plot(anc_dates, anc_sub['anc_pct'],
                color='purple', lw=1.5, ls='-', label='ANC data')

    # ANC mean as dashed reference
    ax.axhline(fac_anc_rank.get(fac, np.nan), color='purple',
               lw=0.7, ls=':', alpha=0.5)

    short = (fac[:22] + '…') if len(fac) > 22 else fac
    ax.set_title(f'{short}\n({group}, ANC≈{fac_anc_rank.get(fac, 0):.1f}%)',
                 fontsize=5.5, pad=2, color=title_colour, fontweight='bold')
    ax.grid(True, alpha=0.2, ls=':')
    ax.tick_params(labelsize=5)
    ax.xaxis.set_major_locator(mdates.YearLocator(5))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right', fontsize=5)
    if i == 0:
        ax.legend(fontsize=5, loc='upper left', framealpha=0.7)

# Hide unused subplots
for j in range(len(panel_facs), len(axes_flat)):
    axes_flat[j].set_visible(False)

fig.supylabel('% appointments disrupted', fontsize=9, x=0.0)
fig.suptitle(
    f'Facility time series — TLO model (blue, IQR band) vs ANC data (purple)\n'
    f'Top {N_PANEL} highest-disrupted (red titles) & {N_PANEL} lowest-disrupted (blue titles) by ANC estimate',
    fontsize=11, fontweight='bold'
)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'facility_timeseries_top_bottom.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Summary statistics & residual diagnostics

In [ ]:
# ── Overall summary ───────────────────────────────────────────────────────────
comp = comparison.dropna(subset=['mean', 'anc_pct']).copy()
comp['residual']    = comp['mean'] - comp['anc_pct']
comp['abs_residual'] = comp['residual'].abs()

print('═' * 55)
print(f'  Facilities matched:          {comp["RealFacility_ID"].nunique():>6}')
print(f'  Facility-year rows:          {len(comp):>6}')
print(f'  Mean ANC disruption:         {comp["anc_pct"].mean():>6.2f}%')
print(f'  Mean TLO disruption:         {comp["mean"].mean():>6.2f}%')
print(f'  Mean residual (TLO − ANC):   {comp["residual"].mean():>6.2f}%')
print(f'  Median |residual|:            {comp["abs_residual"].median():>5.2f}%')
print(f'  Pearson r:                   {comp["mean"].corr(comp["anc_pct"]):>6.3f}')
print(f'  Spearman ρ:                  {comp["mean"].corr(comp["anc_pct"], method="spearman"):>6.3f}')
print('═' * 55)

# ── By facility type ──────────────────────────────────────────────────────────
comp['broad_type'] = comp['Facility_Type'].apply(_broad_type)
by_type = (
    comp.groupby('broad_type')
    .agg(
        n_fac        = ('RealFacility_ID', 'nunique'),
        mean_anc     = ('anc_pct',  'mean'),
        mean_tlo     = ('mean',     'mean'),
        mean_resid   = ('residual', 'mean'),
        median_resid = ('residual', 'median'),
    )
    .round(2)
)
print('\nBy facility type:')
print(by_type.to_string())

# ── By year ───────────────────────────────────────────────────────────────────
by_year = (
    comp.groupby('Year')
    .agg(
        mean_anc   = ('anc_pct',  'mean'),
        mean_tlo   = ('mean',     'mean'),
        mean_resid = ('residual', 'mean'),
    )
    .round(2)
)
print('\nBy year:')
print(by_year.to_string())

In [ ]:
# ── Residual distribution by facility type and over time ─────────────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Left: residual boxplot by facility type
ax = axes[0]
groups_ordered = ['Health Post', 'Health Centre', 'Hospital', 'Other']
data_for_box   = [comp.loc[comp['broad_type'] == g, 'residual'].dropna()
                  for g in groups_ordered]
bp = ax.boxplot(data_for_box, labels=groups_ordered, patch_artist=True,
                medianprops=dict(color='black', lw=2))
for patch, colour in zip(bp['boxes'],
                         ['#1f77b4', '#ff7f0e', '#2ca02c', '#9467bd']):
    patch.set_facecolor(colour)
    patch.set_alpha(0.6)
ax.axhline(0, color='k', lw=1, ls='--', alpha=0.5)
ax.set_ylabel('TLO − ANC (percentage points)', fontsize=10)
ax.set_title('Residuals by facility type', fontsize=11)
ax.grid(True, alpha=0.3, ls=':')

# Right: mean residual over time
ax = axes[1]
mean_resid_year = by_year['mean_resid']
ax.bar(mean_resid_year.index, mean_resid_year.values,
       color=['#d62728' if v > 0 else '#1f77b4' for v in mean_resid_year.values],
       alpha=0.75, edgecolor='white', linewidth=0.5)
ax.axhline(0, color='k', lw=1)
ax.set_xlabel('Year', fontsize=10)
ax.set_ylabel('Mean residual (TLO − ANC, pp)', fontsize=10)
ax.set_title('Mean residual over time\n(red = TLO over-estimates, blue = under-estimates)',
             fontsize=11)
ax.grid(True, alpha=0.3, ls=':',  axis='y')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

fig.suptitle('Residual analysis: TLO model vs ANC disruption data',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'facility_residual_diagnostics.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Export comparison table ───────────────────────────────────────────────────
export_cols = ['RealFacility_ID', 'Year', 'District', 'Facility_Type', 'broad_type',
               'mean', 'median', 'p25', 'p75', 'p05', 'p95',
               'anc_pct', 'residual', 'abs_residual']
export_df = comp[export_cols].round(4)
export_path = OUTPUT_DIR / 'facility_level_comparison_table.csv'
export_df.to_csv(export_path, index=False)
print(f'Saved comparison table → {export_path}')
export_df.head()